In [2]:
import pandas as pd
import numpy as np
import os
import imp
import telephone_analysis
import srilm
import roark
import glob
import shutilwor
import scipy.stats
imp.reload(telephone_analysis)
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import kenlm

In [3]:
import glob

from os.path import join, exists
import pickle

import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertForMaskedLM, BertTokenizer, BartForConditionalGeneration, BartTokenizer

import new_models
from new_models import prep_probs

import load_runs

In [6]:
all_runs = load_runs.load_runs()

all_runs.loc[all_runs.user != "0"].shape[0]
len(np.unique(all_runs.loc[all_runs.user != "0"].user))

0    0
1    5
2    4
3    3
4    1
Name: index, dtype: int64


/home/nwong/chompsky/serial_chain/telephone-analysis-public/load_runs.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  all_runs = pd.concat(dfs)


266

# Word-Level Language Modeling Results

In [44]:
# index in the LM corresponds to the response: user_candidate_transcription. 3192: last input

In [5]:
lm = {}

In [6]:
bnc_knn_lm = srilm.LM("LMs/BNC_merged.LM", lower=True)

In [8]:
lm['bnc_unigram'] =  [telephone_analysis.getSRILMprob(x, {}, bnc_knn_lm, mode='single_word', unigram=True) for x in all_runs['user_candidate_transcription']]

In [9]:
lm['bnc_trigram'] =  [telephone_analysis.getSRILMprob(x, {}, bnc_knn_lm, mode='single_word', unigram = False) for x in all_runs['user_candidate_transcription']]

In [10]:
lm['roark_scores'] = roark.parse([str(i) for i in all_runs['user_candidate_transcription']], 
    '/home/nwong/utils/incremental-top-down-parser',
                          numWorkers=24, mode='single_word')

Input contains 3193 sentences
Input contains 1982 unique sentences
Finished external parsing


/home/nwong/chompsky/serial_chain/telephone-analysis-public/roark.py:235: FutureWarning: read_table is deprecated, use read_csv instead.
  pfix_df = pd.read_table(StringIO('\n'.join(y)), sep=' ', header=None, names = ['prefix','word']+colnames)
/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/pandas/io/parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [12]:
lm['big_lm_scores'] = telephone_analysis.getBigLMscores(all_runs['user_candidate_transcription'], 'big_lm',
    'big_lm_cache', colname='BigLM_probability', lm_1b_dir='/home/nwong/python/lm_1b', mode='single_word')

Big LM call:
cd /home/nwong/python/lm_1b && source bin/activate && bazel-bin/lm_1b/lm_1b_eval --mode eval_sentences --pbtxt data/graph-2016-09-10.pbtxt --vocab_file data/vocab-2016-09-10.txt --eval_dir /home/nwong/chompsky/serial_chain/telephone-analysis-public/big_lm --ckpt 'data/ckpt-*'
2


In [138]:
m = kenlm.Model('LMs/deepspeech_5gram.binary')
lm['kenlm_scores'] = [telephone_analysis.getKenLMProb(x, m, mode='single_word') for x in all_runs['user_candidate_transcription']]

# For consistency in the filtering.
lm['kenlm_scores'] = [df.rename(columns = {'words' : 'word'}) for df in lm['kenlm_scores']] 

In [33]:

for lm_name in lm:

    filtered_df_list = list(map(prep_probs.filter_eos_punct_df, lm[lm_name]))
    lm[lm_name] = filtered_df_list
    # Filter out all of the </S> and "."
    #   before the word comparison, to ensure alignment.
    
    # Note that for this to work with kenlm, the "words" column must be renamed to "word".
    # Additionally there can't be a second "kenlm_probability" column.
    assert(len(lm[lm_name]) == all_runs.shape[0])
    

bnc_unigram
bnc_trigram
roark_scores
big_lm_scores
kenlm_scores


In [34]:
RESULTS_FOLDER = './intermediate_results/new_models_probs'

from new_models import prep_probs
import importlib
import align_prep_words

importlib.reload(prep_probs)

model_names = ['gpt2_normal', 'gpt2_medium', 'bert', 'bart']

tokenizers = {
    'gpt2': GPT2Tokenizer.from_pretrained('gpt2'),
    'bert': BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking"),
    'bart': BartTokenizer.from_pretrained("facebook/bart-base"),
}


In [ ]:
# Note: bnc_unigram words are listed as the "words" of the transformers,
# because of the filtering on bad tokens.

# this means that this also filters out the capitalization, since none of the bnc_unigram_words are capitalized.

word_list = [df['word'].values.tolist() for df in lm['bnc_unigram']]

transformer_names = ['gpt2_normal', 'gpt2_medium', 'bert', 'bart']
tokenizer_names = ['gpt2', 'gpt2', 'bert', 'bart']

for model_name, tokenizer_name in zip(transformer_names, tokenizer_names):

    print(f'Processing model name: {model_name}')
    
    raw_scores = prep_probs.load_word_scores(model_name, RESULTS_FOLDER) # This is not filtering properly.
    lm[f'{model_name}_scores'] = align_prep_words.align_model_word_dfs(raw_scores,
                                                                       tokenizers[tokenizer_name],
                                                                       word_list)

In [39]:
# Need to pickle all of the results

DATA_PREP_FOLDER = './intermediate_results/data_prep_logistic'
if not os.path.exists(DATA_PREP_FOLDER):
    os.makedirs(DATA_PREP_FOLDER)
    
for lm_name, lm_data in lm.items():
    print(lm_name)
    load_runs.pickle_logistic_prep(lm_data, lm_name, DATA_PREP_FOLDER)

bnc_unigram
Saved per-word scores to ./intermediate_results/data_prep_logistic/bnc_unigram_predictions.txt.
bnc_trigram
Saved per-word scores to ./intermediate_results/data_prep_logistic/bnc_trigram_predictions.txt.
roark_scores
Saved per-word scores to ./intermediate_results/data_prep_logistic/roark_scores_predictions.txt.
big_lm_scores
Saved per-word scores to ./intermediate_results/data_prep_logistic/big_lm_scores_predictions.txt.
kenlm_scores
Saved per-word scores to ./intermediate_results/data_prep_logistic/kenlm_scores_predictions.txt.
gpt2_normal_scores
Saved per-word scores to ./intermediate_results/data_prep_logistic/gpt2_normal_scores_predictions.txt.
gpt2_medium_scores
Saved per-word scores to ./intermediate_results/data_prep_logistic/gpt2_medium_scores_predictions.txt.
bert_scores
Saved per-word scores to ./intermediate_results/data_prep_logistic/bert_scores_predictions.txt.
bart_scores
Saved per-word scores to ./intermediate_results/data_prep_logistic/bart_scores_predictio

# Proceed from saved results here

In [7]:
DATA_PREP_FOLDER = './intermediate_results/data_prep_logistic'

model_names = [filename.split('logistic/')[1].split('_predictions.txt')[0]
               for filename in glob.glob(DATA_PREP_FOLDER+'/*')]

lm = {}

for lm_name in model_names:
    raw_scores_path = join(DATA_PREP_FOLDER, f"{lm_name}_predictions.txt")
    # 3/27: https://stackoverflow.com/questions/27745500/how-to-save-a-list-to-a-file-and-read-it-as-a-list-type
    with open(raw_scores_path, 'rb') as f:
        raw_scores = pickle.load(f)
        lm[lm_name] = raw_scores

In [8]:
# sanity check: all sentences in all models must have the same number of tokens in the dataframe
languageModelNames = ['bnc_unigram', 'bnc_trigram', 'roark_scores', 'big_lm_scores', 'kenlm_scores',
                     'gpt2_normal_scores', 'gpt2_medium_scores', 'bert_scores', 'bart_scores']

for i in range(all_runs.shape[0]):
    sentences = [lm[x][i] for x in languageModelNames]
    try:
        numWords = [x.shape[0] for x in sentences]
    except:
        print('Problem counting words')
        import pdb
        pdb.set_trace()
    if not np.allclose(numWords[1:len(numWords)], numWords[0]):
        print('Different number of words')
        import pdb
        pdb.set_trace()

In [9]:
sentence_store = []
for i in range(all_runs.shape[0]):
    # need to rename what is coming out of each language model
    word_store = [] 
    for languageModelName in languageModelNames:
        df = lm[languageModelName][i].copy()
        try:
            df.columns = [languageModelName+'_'+x for x in df.columns]        
        except:
            import pdb
            pdb.set_trace()
        word_store.append(df)
    lms_combined = pd.concat(word_store, axis =1)
    lms_combined['sCounter'] = range(lms_combined.shape[0])
    lms_combined['sentence_index'] = i
    sentence_store.append(lms_combined)
wdf = pd.concat(sentence_store)

In [10]:
wdf['word'] = wdf[u'bnc_unigram_word']

In [11]:
wdf.iloc[0:21] # Nonfiction (two versions: one with non fiction, one with nonfiction)

#wdf.iloc[170:200] # Dietitian index.

,bnc_unigram_index,bnc_unigram_prob,bnc_unigram_word,bnc_trigram_index,bnc_trigram_prob,bnc_trigram_word,roark_scores_prefix,roark_scores_word,roark_scores_prefix.1,roark_scores_srprsl,...,gpt2_normal_scores_word,gpt2_medium_scores_prob,gpt2_medium_scores_word,bert_scores_prob,bert_scores_word,bart_scores_prob,bart_scores_word,sCounter,sentence_index,word
0,0,-3.310068,each,0,-2.994068,each,pfix:1,each,6.566,2.851578,...,each,-3.225354,each,-0.326024,each,-1.289798,each,0,0,each
1,1,-7.309903,nonfiction,1,-8.412464,nonfiction,pfix:2,nonfiction,18.290,5.091669,...,nonfiction,NaN,nonfiction,-3.985277,nonfiction,NaN,nonfiction,1,0,nonfiction
2,2,-3.631093,book,2,-3.756868,book,pfix:3,book,28.519,4.442398,...,book,-0.165244,book,-2.205348,book,-0.226925,book,2,0,book
3,3,-2.600407,has,3,-2.253614,has,pfix:4,has,32.402,1.686800,...,has,-1.251818,has,-0.091852,has,-0.815807,has,3,0,has
4,4,-1.681165,a,4,-1.317157,a,pfix:5,a,35.256,1.239042,...,a,-0.487444,a,-0.025494,a,-0.135504,a,4,0,a
5,5,-3.737589,call,5,-3.763689,call,pfix:6,call,44.218,3.892147,...,call,-3.728456,call,-2.599102,call,-3.694493,call,5,0,call
6,6,-3.321531,number,6,-3.708286,number,pfix:7,number,53.366,3.972926,...,number,-2.610043,number,-1.608924,number,-3.173203,number,6,0,number
7,7,-2.162270,on,7,-2.029635,on,pfix:8,on,58.319,2.151495,...,on,-1.057074,on,-0.028152,on,-0.540727,on,7,0,on
8,8,-2.804867,its,8,-2.394903,its,pfix:9,its,62.220,1.693748,...,its,-0.773916,its,-0.411839,its,-0.597893,its,8,0,its
9,9,-5.034822,spine,9,-4.401074,spine,pfix:10,spine,77.301,6.549595,...,spine,-1.306274,spine,-2.007116,spine,-2.149205,spine,9,0,spine


In [ ]:
# Each record is a 'produced' word (user_candidate_transcription) 

# Identifying Deleted and Changed Words

In [ ]:
# for generation n, get the indices of all words that have changed in n+1
# have changed: no longer appear? doesn't handle transpositions
# borrowed the function from the old version of telephone

In [15]:
# Code from the Aggregate Notebook, this prevents an error that says that a "float" is being encoded (i.e. NaN)
for x in ['condition','flag_type','gold_candidate_transcription',
          'gold_comparison_transcription','reason', 'subject_id',
         'upstream_subject_id', 'user_comparison_transcription', 'user_short']:
    all_runs[x] = all_runs[x].fillna(value="none")

In [16]:
all_runs.loc[all_runs.user == "0"].head(5)

,chain,character_levdau,check_time,condition,flag_type,gold_candidate_transcription,gold_comparison_transcription,gold_dist,length_accept,reason,...,upstream_pointer,upstream_subject_id,user,user_candidate_transcription,user_comparison_transcription,user_dist,user_short,word_distances,unique_chain_identifier,global_chain
0,0,NaN,NaN,none,none,none,none,NaN,NaN,none,...,-1,none,0,each nonfiction book has a call number on its ...,none,NaN,none,NaN,180419_AMT_lengthLimitedGPU_0,0
18,0,NaN,NaN,none,none,none,none,NaN,NaN,none,...,-1,none,0,a dietitian goes to college for at least four ...,none,NaN,none,NaN,180419_AMT_lengthLimitedGPU_0,0
43,0,NaN,NaN,none,none,none,none,NaN,NaN,none,...,-1,none,0,the iris absorbs all of the light waves except...,none,NaN,none,NaN,180419_AMT_lengthLimitedGPU_0,0
66,0,NaN,NaN,none,none,none,none,NaN,NaN,none,...,-1,none,0,some acids you may know are vinegar and lemon ...,none,NaN,none,NaN,180419_AMT_lengthLimitedGPU_0,0
93,0,NaN,NaN,none,none,none,none,NaN,NaN,none,...,-1,none,0,the chase leads across a field toward a nearby...,none,NaN,none,NaN,180419_AMT_lengthLimitedGPU_0,0


In [17]:
# take a 2-column subset of all_trials that we can put into R
input_output = all_runs[['gold_candidate_transcription','user_candidate_transcription','user',
                         'upstream_subject_id']]
#gold_candidate_transcription is what a participant heard
#user_candidate_transcription is what the participant produced

# remove the intitial sentences -- these are represnted as input for the first participant
%R -i input_output

/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "gold_candidate_transcription". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  % (name, str(e)))
/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "user_candidate_transcription". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  % (name, str(e)))
/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "user". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  % (name, str(e)))
/home/nwong/chompsky/serial_chain/telephone-analy

In [18]:
input_output

,gold_candidate_transcription,user_candidate_transcription,user,upstream_subject_id
0,none,each nonfiction book has a call number on its ...,0,none
1,each nonfiction book has a call number on its ...,each non fiction book has a call number in its...,8cf6535ea0ae4addb28f5f90a2b13a7d,0
2,each non fiction book has a call number in its...,each nonfiction book had a call number on its ...,edf8f705ab7241a191fa3ae24e381ebc,8cf6535ea0ae4addb28f5f90a2b13a7d
3,Each nonfiction book had a call number on its ...,each nonfiction book had a call number on its ...,5bcd571bb9b645bc92fe6a4a7d5cfb35,edf8f705ab7241a191fa3ae24e381ebc
4,each nonfiction book had a call number on its ...,each non fiction book had a call number on its...,d14ebf1fffae49d8b76323eb6c393120,5bcd571bb9b645bc92fe6a4a7d5cfb35
5,each non fiction book had a call number on its...,each nonfiction book had a call number on its ...,67497f6ffb7e47d795ea7b655ccdd0ca,d14ebf1fffae49d8b76323eb6c393120
6,each nonfiction book had a call number on its ...,each non fiction book had 12 numbers on its spine,dc02e8bc38234eed94e4e0cfbe083801,67497f6ffb7e47d795ea7b655ccdd0ca
7,Each non fiction book had 12 numbers on its spine,each nonfiction book had twelve numbers on its...,e1f5fb8bc21048cb8a50771b80eda39a,dc02e8bc38234eed94e4e0cfbe083801
8,Each nonfiction book had twelve numbers on its...,each non fiction book had twelve numbers on it...,53edaed0aee2422e96af6c0d3d636962,e1f5fb8bc21048cb8a50771b80eda39a
9,each non fiction book had twelve numbers on it...,each nonfiction book had twelve notches on its...,a575811edc0841b082724370f678d574,53edaed0aee2422e96af6c0d3d636962


In [19]:
%%R
source('getWordLevenshteinDistance.R')
print(paste(nrow(input_output), 'sentences'))
names(input_output) = c('input','output','output_subject','input_subject')  
input_output$input = tolower(as.character(input_output$input))
input_output$output = tolower(as.character(input_output$output))

computeEditTable = function(s,r,input_subject, output_subject){
    if (s == 'none'){
        # this is an initial sentence, return NA
        return(NA)
    } else {
        et = getReducedEditTable(s,r)
        # for python compatibility, use 0-indices
        et$sCounter = et$sCounter - 1
        et$rCounter = et$rCounter - 1
        et$input_subject = input_subject
        et$output_subject = output_subject
        return(et)
    }
}

editTables = mapply(computeEditTable, input_output$input, input_output$output,
                   input_output$input_subject, input_output$output_subject)

[1] "3193 sentences"


In [20]:
%%R
editTables[[2]] #this should correspond to wdf[,1]

   code      sWord   rWord sCounter rCounter
1     M       each    each        0        0
2     D nonfiction    <NA>        1       NA
3     I       <NA>     non       NA        1
4     I       <NA> fiction       NA        2
5     M       book    book        2        3
6     M        has     has        3        4
7     M          a       a        4        5
8     M       call    call        5        6
9     M     number  number        6        7
11    S         on      in        7        8
12    M        its     its        8        9
13    M      spine   spine        9       10
                                              sentence
1  each nonfiction book has a call number on its spine
2  each nonfiction book has a call number on its spine
3  each nonfiction book has a call number on its spine
4  each nonfiction book has a call number on its spine
5  each nonfiction book has a call number on its spine
6  each nonfiction book has a call number on its spine
7  each nonfiction book has a 

/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "code". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  % (name, str(e)))
/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "sWord". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'str'> and <class 'rpy2.rinterface_lib.sexp.NACharacterType'>).
  % (name, str(e)))
/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "rWord". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <cla

In [21]:
%%R
print(computeEditTable('this is a', 'this is a mouse', NULL, NULL))
print(computeEditTable('this is a mouse', 'this is a', NULL, NULL ))
print(computeEditTable('this is a mouse', 'this is a mouse', NULL, NULL))
print(computeEditTable('this is a mouse', 'this is a house', NULL, NULL))

  code sWord rWord sCounter rCounter  sentence        response sLeftSequence
1    M  this  this        0        0 this is a this is a mouse              
2    M    is    is        1        1 this is a this is a mouse              
3    M     a     a        2        2 this is a this is a mouse     this is a
4    I  <NA> mouse       NA        3 this is a this is a mouse              
  rLeftSequence sRightSequence rRightSequence
1                    this is a      this is a
2                                  is a mouse
3     this is a                              
4    is a mouse                              
  code sWord rWord sCounter rCounter        sentence  response sLeftSequence
1    M  this  this        0        0 this is a mouse this is a              
2    M    is    is        1        1 this is a mouse this is a              
3    M     a     a        2        2 this is a mouse this is a     this is a
4    D mouse  <NA>        3       NA this is a mouse this is a    is a mouse


In [22]:
%%R
getDSMwrtInput = function(sentence_index, editTables){
    et = editTables[[sentence_index]]
    if (is.na(et)){
        return(data.frame(sentence_index, sCounter=NA,code=NA, input_subject=NA, output_subject=NA)) # these are initial sentences
    } else{
        et = editTables[[sentence_index]]    
        et$sentence_index = sentence_index
        # here is where insertions are removed so that we can join back with wfds
        return(subset(et, !is.na(sCounter))[,c('sentence_index','sCounter','code','input_subject','output_subject')])
    }
}

DSMwrtInput = do.call('rbind', lapply(c(1:length(editTables)), function(i){
    dsm = getDSMwrtInput(i, editTables)  
    if (!is.na(dsm)){
        dsm$sentence_index = dsm$sentence_index - 2 
        # -1 because Python indexes from 0
        # another -1 to bring the index of edits into alignment with the languag model results     
    } 
    return(dsm)
}))   

#DSMwrtInput = subset(DSMwrtInput, !is.na(code))

In [23]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
DSMwrtInput = r['DSMwrtInput']
DSMwrtInput.iloc[0:15,]

,sentence_index,sCounter,code,input_subject,output_subject
1,-1.0,NaN,NA_character_,NA_character_,NA_character_
110,0.0,0.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
2,0.0,1.0,D,0,8cf6535ea0ae4addb28f5f90a2b13a7d
5,0.0,2.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
6,0.0,3.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
7,0.0,4.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
8,0.0,5.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
9,0.0,6.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
11,0.0,7.0,S,0,8cf6535ea0ae4addb28f5f90a2b13a7d
12,0.0,8.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d


In [24]:
initialSentenceIndices = np.array(DSMwrtInput[np.isnan(DSMwrtInput.sCounter)]['sentence_index'].tolist())
wdf_initRemoved = wdf[(~wdf['sentence_index'].isin(initialSentenceIndices))]
DSMwrtInput_nansRemoved = DSMwrtInput[~np.isnan(DSMwrtInput.sCounter)]

In [25]:
print(DSMwrtInput_nansRemoved.shape[0])
print(wdf_initRemoved.shape[0])
#assert(DSMwrtInput_nansRemoved.shape[0] == wdf_initRemoved.shape[0])

27290
27298


In [26]:
wdf_initRemoved[wdf_initRemoved.sentence_index == 3192]

,bnc_unigram_index,bnc_unigram_prob,bnc_unigram_word,bnc_trigram_index,bnc_trigram_prob,bnc_trigram_word,roark_scores_prefix,roark_scores_word,roark_scores_prefix.1,roark_scores_srprsl,...,gpt2_normal_scores_word,gpt2_medium_scores_prob,gpt2_medium_scores_word,bert_scores_prob,bert_scores_word,bart_scores_prob,bart_scores_word,sCounter,sentence_index,word
0,0,-3.081292,before,0,-3.037731,before,pfix:1,before,6.993,3.037021,...,before,-2.958956,before,-0.451015,before,-0.520920,before,0,3192,before
1,1,-2.186153,you,1,-1.185921,you,pfix:2,you,13.198,2.694363,...,you,-1.037076,you,-0.024068,you,-0.023925,you,1,3192,you
2,2,-3.713525,leave,2,-1.507196,leave,pfix:3,leave,20.940,3.362308,...,leave,-2.037063,leave,-0.618477,leave,-1.712257,leave,2,3192,leave
3,3,-3.121925,make,3,-4.336845,make,pfix:4,make,30.850,4.303858,...,make,-3.606973,make,-0.014824,make,-1.734913,make,3,3192,make
4,4,-3.643197,sure,4,-1.549859,sure,pfix:5,sure,36.383,2.402951,...,sure,-0.028073,sure,-0.003831,sure,-0.119730,sure,4,3192,sure
5,5,-2.186153,you,5,-0.752118,you,pfix:6,you,43.568,3.120406,...,you,-0.334407,you,-0.122452,you,-0.195653,you,5,3192,you
6,6,-3.512815,change,6,-3.870051,change,pfix:7,change,54.143,4.593098,...,change,-2.447857,change,-3.240940,change,-2.139447,change,6,3192,change
7,7,-3.763360,everything,7,-3.264657,everything,pfix:8,everything,63.772,4.181822,...,everything,-2.191092,everything,-1.073453,everything,-2.241529,everything,7,3192,everything


In [27]:
DSMwrtInput_nansRemoved[DSMwrtInput_nansRemoved.sentence_index == 3192] 

,sentence_index,sCounter,code,input_subject,output_subject


In [28]:
dsm_counts = DSMwrtInput_nansRemoved.groupby(['sentence_index']).sCounter.agg(np.size).reset_index()

dsm_counts.columns = ['sentence_index', 'dsm_count']
wdf_counts = wdf_initRemoved.groupby('sentence_index').sCounter.agg(np.size).reset_index()
wdf_counts.columns = ['sentence_index', 'wdf_count']
count_check = wdf_counts.merge(dsm_counts, how='outer')
count_check # differences in the counts?
count_check['difference'] = count_check.wdf_count - count_check.dsm_count
count_check.sort_values(by=['difference'], ascending=False)

,sentence_index,wdf_count,dsm_count,difference
0,0,10,10.0,0.0
2003,2142,9,9.0,0.0
1994,2132,8,8.0,0.0
1995,2133,8,8.0,0.0
1996,2134,8,8.0,0.0
1997,2135,8,8.0,0.0
1998,2136,8,8.0,0.0
1999,2138,10,10.0,0.0
2000,2139,9,9.0,0.0
2001,2140,10,10.0,0.0


In [29]:
# merge DSMwrtInput into the word data frame
wdfr = wdf_initRemoved.merge(DSMwrtInput_nansRemoved) #sCounter is NaN for input sentences

In [30]:
#where did all my happy data go?
wdfr.code.value_counts()

M    22482
D     3442
S     1366
Name: code, dtype: int64

In [31]:
wdfr

,bnc_unigram_index,bnc_unigram_prob,bnc_unigram_word,bnc_trigram_index,bnc_trigram_prob,bnc_trigram_word,roark_scores_prefix,roark_scores_word,roark_scores_prefix.1,roark_scores_srprsl,...,bert_scores_prob,bert_scores_word,bart_scores_prob,bart_scores_word,sCounter,sentence_index,word,code,input_subject,output_subject
0,0,-3.310068,each,0,-2.994068,each,pfix:1,each,6.566,2.851578,...,-0.326024,each,-1.289798,each,0,0,each,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
1,1,-7.309903,nonfiction,1,-8.412464,nonfiction,pfix:2,nonfiction,18.290,5.091669,...,-3.985277,nonfiction,NaN,nonfiction,1,0,nonfiction,D,0,8cf6535ea0ae4addb28f5f90a2b13a7d
2,2,-3.631093,book,2,-3.756868,book,pfix:3,book,28.519,4.442398,...,-2.205348,book,-0.226925,book,2,0,book,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
3,3,-2.600407,has,3,-2.253614,has,pfix:4,has,32.402,1.686800,...,-0.091852,has,-0.815807,has,3,0,has,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
4,4,-1.681165,a,4,-1.317157,a,pfix:5,a,35.256,1.239042,...,-0.025494,a,-0.135504,a,4,0,a,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
5,5,-3.737589,call,5,-3.763689,call,pfix:6,call,44.218,3.892147,...,-2.599102,call,-3.694493,call,5,0,call,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
6,6,-3.321531,number,6,-3.708286,number,pfix:7,number,53.366,3.972926,...,-1.608924,number,-3.173203,number,6,0,number,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
7,7,-2.162270,on,7,-2.029635,on,pfix:8,on,58.319,2.151495,...,-0.028152,on,-0.540727,on,7,0,on,S,0,8cf6535ea0ae4addb28f5f90a2b13a7d
8,8,-2.804867,its,8,-2.394903,its,pfix:9,its,62.220,1.693748,...,-0.411839,its,-0.597893,its,8,0,its,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
9,9,-5.034822,spine,9,-4.401074,spine,pfix:10,spine,77.301,6.549595,...,-2.007116,spine,-2.149205,spine,9,0,spine,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d


# Merge With Word Properties

In [32]:
wdfr['word'] = wdfr['bnc_unigram_word']
lexiconch = pd.read_csv('data/lexiconch.csv', index_col=0)
print('Number of words before merging with Lexiconch: '+str(wdfr.shape[0]))
wdfl = wdfr.merge(lexiconch, how='left')
print('Number of words after merging with Lexiconch: '+str(wdfl.shape[0]))
' '.join(set(wdfr.word) - set(wdfl.word))

/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9,10,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Number of words before merging with Lexiconch: 27290
Number of words after merging with Lexiconch: 27290


''

In [33]:
lexiconch.iloc[0]

word                               roadsweeper
conc_bigram                                  0
conc_conc_m                               4.85
conc_conc_sd                              0.37
conc_unknown                                 1
conc_total                                  27
conc_percent_known                        0.96
conc_subtlex                                 0
conc_dom_pos                                 0
kpm_alternative_spelling          road sweeper
kpm_freq_pm                                NaN
kpm_dom_pos_subtlex                        NaN
kpm_nletters                                11
kpm_nphon                                    8
kpm_nsyll                                    3
kpm_lemma_highest_pos              roadsweeper
kpm_aoa_kup                               9.83
kpm_perc_known                               1
kpm_aoa_kup_lem                           9.83
kpm_perc_known_lem                           1
kpm_aoa_bird_lem                          7.12
kpm_aoa_brist

In [34]:
yarkoni_pld = pd.read_table('data/pld20.txt', header=None)
yarkoni_pld.columns = ['word', 'pld20']

/home/nwong/chompsky/serial_chain/telephone-analysis-public/telephone-env-3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [35]:
print('Number of words before merging with Yarkoni PLD: '+str(wdfl.shape[0]))
wdfy = wdfl.merge(yarkoni_pld, how='left')
print('Number of words after merging with Yarkoni PLD: '+str(wdfy.shape[0]))

Number of words before merging with Yarkoni PLD: 27290
Number of words after merging with Yarkoni PLD: 27290


In [36]:
subtlex = pd.read_csv('data/subtlex_augmented.csv')
subtlex['word'] = subtlex.Word
subtlex.iloc[0]

Word                         the
FREQcount                1501908
CDcount                     8388
FREQlow                  1339811
Cdlow                       8388
SUBTLWF                  29449.2
Lg10WF                    6.1766
SUBTLCD                      100
Lg10CD                    3.9237
ipa_                      [ð, ə]
ipa_single                [ð, ə]
ipa_diphthongs            [ð, ə]
unigram_probability    0.0302076
unigram_surprisal        3.49966
word                         the
Name: 0, dtype: object

In [37]:
print('Number of words before merging with Subtlex: '+str(wdfy.shape[0]))
wdfx = wdfy.merge(subtlex[['word','SUBTLCD']], how='left')
print('Number of words after merging with Subtlex: '+str(wdfx.shape[0]))

Number of words before merging with Subtlex: 27290
Number of words after merging with Subtlex: 27290


# Merge in Sentence Level Predictors

In [51]:
raw_sentence_data_path = './intermediate_results/all_runs_raw_model_outputs.csv'
all_runs_sentence = load_runs.postprocess_runs(raw_sentence_data_path)

all_runs_sentence['sentence_index'] = range(all_runs.shape[0])

Getting initial probabilities for biglm_probability
Getting initial probabilities for WSJ_Roark_Negative.Log.Probability
Getting initial probabilities for BNC_KNN_unigramProb
Getting initial probabilities for BNC_KNN_trigramProb
Getting initial probabilities for kenlm_probability
Getting initial probabilities for bllip_probability
Getting initial probabilities for bllip_wsj_probability
Getting initial probabilities for mikolov_wsj_probability
Getting initial probabilities for WSJ_gt_unigramProb
Getting initial probabilities for WSJ_gt_trigramProb
Getting initial probabilities for WSJ_gt_5gramProb
Getting initial probabilities for gpt2_normal_probability
Getting initial probabilities for gpt2_medium_probability
Getting initial probabilities for bert_probability
Getting initial probabilities for bart_probability


In [52]:
wdfs = wdfx.merge(all_runs_sentence, on='sentence_index')

In [53]:
wdfx.columns

Index(['bnc_unigram_index', 'bnc_unigram_prob', 'bnc_unigram_word',
       'bnc_trigram_index', 'bnc_trigram_prob', 'bnc_trigram_word',
       'roark_scores_prefix', 'roark_scores_word', 'roark_scores_prefix.1',
       'roark_scores_srprsl', 'roark_scores_SynSp', 'roark_scores_LexSp',
       'roark_scores_ambig', 'roark_scores_open', 'roark_scores_rernk',
       'roark_scores_toprr', 'roark_scores_stps', 'big_lm_scores_Unnamed: 0',
       'big_lm_scores_prob', 'big_lm_scores_word', 'kenlm_scores_prob',
       'kenlm_scores_preceding', 'kenlm_scores_unk', 'kenlm_scores_word',
       'gpt2_normal_scores_prob', 'gpt2_normal_scores_word',
       'gpt2_medium_scores_prob', 'gpt2_medium_scores_word',
       'bert_scores_prob', 'bert_scores_word', 'bart_scores_prob',
       'bart_scores_word', 'sCounter', 'sentence_index', 'word', 'code',
       'input_subject', 'output_subject', 'conc_bigram', 'conc_conc_m',
       'conc_conc_sd', 'conc_unknown', 'conc_total', 'conc_percent_known',
       'c

In [59]:
wdfs.loc[wdfs.sentence_index == 1][['bnc_unigram_word',
                                   'user_candidate_transcription',
                                    'gold_candidate_transcription',
                                    'input_subject',
                                    'output_subject'
                                   ]]

,bnc_unigram_word,user_candidate_transcription,gold_candidate_transcription,input_subject,output_subject
10,each,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
11,non,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
12,fiction,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
13,book,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
14,has,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
15,a,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
16,call,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
17,number,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
18,in,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
19,its,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc


# Save the output

In [68]:
wdfs.to_csv('output/wordLevelChanges.csv', index=False)

In [65]:
for col in wdfs.columns:
    print(col)

bnc_unigram_index
bnc_unigram_prob
bnc_unigram_word
bnc_trigram_index
bnc_trigram_prob
bnc_trigram_word
roark_scores_prefix
roark_scores_word
roark_scores_prefix.1
roark_scores_srprsl
roark_scores_SynSp
roark_scores_LexSp
roark_scores_ambig
roark_scores_open
roark_scores_rernk
roark_scores_toprr
roark_scores_stps
big_lm_scores_Unnamed: 0
big_lm_scores_prob
big_lm_scores_word
kenlm_scores_prob
kenlm_scores_preceding
kenlm_scores_unk
kenlm_scores_word
gpt2_normal_scores_prob
gpt2_normal_scores_word
gpt2_medium_scores_prob
gpt2_medium_scores_word
bert_scores_prob
bert_scores_word
bart_scores_prob
bart_scores_word
sCounter
sentence_index
word
code
input_subject
output_subject
conc_bigram
conc_conc_m
conc_conc_sd
conc_unknown
conc_total
conc_percent_known
conc_subtlex
conc_dom_pos
kpm_alternative_spelling
kpm_freq_pm
kpm_dom_pos_subtlex
kpm_nletters
kpm_nphon
kpm_nsyll
kpm_lemma_highest_pos
kpm_aoa_kup
kpm_perc_known
kpm_aoa_kup_lem
kpm_perc_known_lem
kpm_aoa_bird_lem
kpm_aoa_bristol_lem
kp

## Investigating the "missing observations"

In [ ]:
"bert_scores_prob missing 422 observations"
"gpt2_normal_scores_prob missing 781 observations"
"gpt2_medium_scores_prob missing 781 observations"
"bart_scores_prob missing 781 observations"

In [67]:
for name in ['bert_scores_prob', 'gpt2_normal_scores_prob', 'gpt2_medium_scores_prob', 'bart_scores_prob']:
    curr_obs = wdfs[name]
    this_sum = sum(pd.isnull(curr_obs))
    print(this_sum)

422
781
781
781


See R notebook, Error Prediction Model - Logistic Regression